# Bayesian Logistic Regression

In [411]:
include("julia/bayes_logreg.jl");
include("julia/optimization.jl");
using CSV;
using DataFrames;
df = CSV.read("data/cats_dogs.csv", DataFrame);
rename!(df, [:V1, :V2] .=> [:x_1, :x_2]);

For plotting we will use the R package `ggplot2` which can be readily integrated into Julia (just like many other R packages, see for example [here](https://avt.im/blog/2018/03/23/R-packages-ggplot-in-julia#fn:rcall))

In [412]:
using RCall
@rlibrary ggplot2
@rlibrary emojifont
p = ggplot() +
  theme_bw();

In [413]:
y = df[:,:y];
N = length(y);
X = Matrix(df[:,Not(:y)]);
X = [ones(N) X]; # add for constant
d = size(X)[2]; # number of features
w_0 = zeros(d);

In [414]:
H_0 = UniformScaling(1/1e10);
mle = bayes_logreg(X,y,w_0,H_0,𝓁,∇𝓁,∇∇𝓁);

## Posterior 

In [415]:
step = 0.05;
range = 2;
W = expandgrid(-range:step:range,-range:step:range);
N_grid = size(W)[1];
σ = [1e-10, 1, 100, 1e10]; # prior uncertainty

In [416]:
estimates = DataFrame()
for i=1:length(σ)
    H_0 = UniformScaling(1/(σ[i]^2));
    model = bayes_logreg(X,y,w_0,H_0,𝓁,∇𝓁,∇∇𝓁);
    posterior = -1 .* [logpdf(MvNormal(model.μ,model.Σ),vcat(model.μ[1],W[n,:])) for n=1:N_grid]
    estimate = DataFrame(hcat(posterior,W), :auto);
    insertcols!(estimate, 4, :noise => σ[i])
    estimates = vcat(estimates, estimate)
end 
transform!(groupby(estimates, :noise), :x1 => (x -> x ./ maximum(x)) => :x1);

In [417]:
points = DataFrame(x=mle.μ[2],y=mle.μ[3]);

In [418]:
p + 
  geom_contour_filled(data=estimates, aes(x = :x2, y = :x3, z = :x1), bins=25, alpha=0.8) +
  geom_point(data=points, aes(x=:x, y=:y), size=3, colour="coral") +
  scale_x_continuous(expand = R"c(0, 0)") + scale_y_continuous(expand = R"c(0, 0)") +
  guides(fill="none") +
  facet_wrap(R"~noise") +
  labs(x="Height (coefficient)", y="Tail (coefficient)") |>
  p -> ggsave("www/posterior.png", plot = p, width=5, height=5)

RObject{StrSxp}
[1] "www/posterior.png"


## Posterior Predictive

In [419]:
step = 1;
expand = 20;
W = expandgrid((minimum(df.x_1)-expand):step:(maximum(df.x_1)+expand),(minimum(df.x_2)-expand):step:(maximum(df.x_2)+expand));
N_grid = size(W)[1];
σ = [1e-10, 1, 100, 1e10]; # prior uncertainty

In [420]:
features = DataFrame()
estimates = DataFrame()
for i=1:length(σ)
    H_0 = UniformScaling(1/(σ[i]^2));
    model = bayes_logreg(X,y,w_0,H_0,nll,∇,∇∇);
    w_map = model.μ;
    y_probit = posterior_predictive(model, hcat(ones(N_grid), W));
    estimate = DataFrame(hcat(y_probit,W), :auto);
    insertcols!(estimate, 4, :noise => σ[i])
    estimates = vcat(estimates, estimate)
    feature = copy(df)
    insertcols!(feature, :noise => σ[i])
    features = vcat(features, feature)
    end 
# Adding emojis 
transform!(features, :y => (x -> ifelse.(x.==0,"www/cat.png","www/dog.png"))  => :emoji);

In [421]:
p + 
  geom_contour_filled(data=estimates, aes(x = :x2, y = :x3, z = :x1), bins=25, alpha=0.8) +
  geom_image(data=features, aes(x=:x_1, y=:x_2, image=:emoji), size=0.1) +
  guides(fill="none") +
  scale_x_continuous(expand = R"c(0, 0)") + scale_y_continuous(expand = R"c(0, 0)") +
  facet_wrap(R"~noise") +
  labs(x="Height", y="Tail") |>
  p -> ggsave("www/predictive.png", plot = p, width=5, height=5)


RObject{StrSxp}
[1] "www/predictive.png"
